jitteringをかけたfrcnn featureのnumpy arrayとphrase pairのデータセットのalignment

In [1]:
% cd /home/mayu-ot/durga/Experiments/loc_iparaphrasing/

/mnt/fs1/mayu-ot/Experiments/loc_iparaphrasing


In [2]:
import numpy as np
from func.datasets.datasets import BBoxDataset, iParaphraseDataset
from func.datasets.converters import cvrt_bbox

/home/mayu-ot/miniconda3/envs/py36chainer/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
bbox_data = BBoxDataset('val')

In [4]:
bbox_data.df.head()

ymin  xmin  ymax  xmax  n_id
image  org_phrase                                     
301246 A group of men       72    92   257   495     0
       A group of people    72    92   257   495     1
       Dock workers         72    92   257   495     2
       Three men            72   130   257   480     3
       Workers              72    92   257   495     4

In [5]:
class PPairDataset(iParaphraseDataset):
    def __init__(self, split, san_check=False):
        super(PPairDataset, self).__init__(split, san_check)
        self.bbox_df = BBoxDataset(split).df
        self.vis_feat = np.load('data/region_feat/jitter_roi-frcnn/%s.npy' % split)
        
    def get_feat_row_id(self, i):
        img_id = self.pair_data.iloc[i][['image']].values[0]
        phr1, phr2 = self.pair_data.iloc[i][['original_phrase1', 'original_phrase2']]
        nid_1 = self.bbox_df.loc[(img_id, phr1)].n_id
        nid_2 = self.bbox_df.loc[(img_id, phr2)].n_id
        return nid_1, nid_2
    
    def get_bbox(self, i):
        img_id = self.pair_data.iloc[i][['image']].values[0]
        phr1, phr2 = self.pair_data.iloc[i][['original_phrase1', 'original_phrase2']]
        bbox_1 = self.bbox_df.loc[(img_id, phr1)][['ymin', 'xmin', 'ymax', 'xmax']]
        bbox_2 = self.bbox_df.loc[(img_id, phr2)][['ymin', 'xmin', 'ymax', 'xmax']]
        return bbox_1, bbox_2
    
    def get_vis_feat(self, i):
        nid_1, nid_2 = self.get_feat_row_id(i)
        return self.vis_feat[nid_1], self.vis_feat[nid_2]
        
    def get_example(self, i):
        xvis_1, xvis_2 = self.get_vis_feat(i)
        img = self.read_image(i)
        phr_1, phr_2 = self.get_phrase_feat(i)
        l = self.pair_data.iloc[i][['ytrue']].values[0]
        
        return img, phr_1, phr_2, xvis_1, xvis_2, l

In [7]:
p_data = PPairDataset('val', san_check=True)

val data: 81 pairs


In [12]:
img, phr_1, phr_2, xvis_1, xvis_2, l =p_data[80]

In [13]:
l

False